## [Eng de Dados] Trabalho Final - DataOps
#### Benício B. de A. Carneiro
10/05/2025


In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
import pandas as pd
import sqlite3



### [INGESTAO] Chamada da API

In [2]:
data = []

In [3]:
response = requests.get('https://api-dados-abertos.cearatransparente.ce.gov.br/transparencia/servidores/salarios?year=2024&month=4&page=1',
                        headers={'accept':'application/json'}).json()

In [4]:
total_pages = 2 #response['sumary']['total_pages']

In [5]:
def get_data(i):
    response = requests.get(f'https://api-dados-abertos.cearatransparente.ce.gov.br/transparencia/servidores/salarios?year=2024&month=1&page={i}',
            headers={'accept':'application/json'}).json()
    return response['data']

In [6]:
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(get_data, i): i for i in range(1,total_pages+1)}
    [data.extend(future.result()) for future in as_completed(futures)]

In [7]:
df_bronze = pd.DataFrame(data)

# Create a connection to the SQLite database
conn_sqlite = sqlite3.connect('salarios-servidores.db')

# Save the DataFrame to the SQLite database
df_bronze.to_sql('landing-abril', conn_sqlite, if_exists='replace', index=False)


# # Close the connection
# conn_sqlite.close()

200